In [57]:
import arcpy
import os
import pandas as pd
import numpy as np

In [75]:
# Get the project root (two levels up from notebook location)
project_root = os.path.dirname(os.path.dirname(os.getcwd())) 

# Set paths using relative structure
gdb_path = os.path.join(project_root, "WebGisProject", "MyProject8", "MyProject8.gdb")
county_shapefile = os.path.join(project_root, "Data", "counties","Counties_-_OSi_National_Statutory_Boundaries_-_2019", "Counties___OSi_National_Statutory_Boundaries_.shp")
weather_csv = os.path.join(project_root,"Data", "county_weather.csv") 
yearly_weather_csv = os.path.join(project_root,"Data", "county_yearly_weather.csv") 
counties_with_weather=os.path.join(project_root,"Data", "counties_with_weather") 

In [76]:
print(f"Absolute path: {os.path.abspath(weather_csv)}")
print(f"Absolute path: {os.path.abspath(yearly_weather_csv)}")
print(f"Absolute path: {os.path.abspath(counties_with_weather)}")

Absolute path: C:\Users\asus\Desktop\Project\Data\county_weather.csv
Absolute path: C:\Users\asus\Desktop\Project\Data\county_yearly_weather.csv
Absolute path: C:\Users\asus\Desktop\Project\Data\counties_with_weather


In [37]:
#Reading and dispalying weather_csv table
df_weather = pd.read_csv(weather_csv)
df_weather.head()

,COUNTY,Year,Avg_rain(mm),Avg_temp (C),Avg_wind(knots)
0,CARLOW,2020-2024,82.47,10.59,7.37
1,CAVAN,2020-2024,105.91,9.97,5.94
2,CLARE,2020-2024,115.94,11.08,8.51
3,CORK,2020-2024,121.08,0.00,0.00
4,DONEGAL,2020-2024,131.27,10.42,10.15


In [38]:
#Reading and dispalying yearly_weather_csv table
df_year_weather = pd.read_csv(yearly_weather_csv)
df_year_weather.head()

,COUNTY,Year,Avg_rain(mm),Avg_temp (C),Avg_wind(knots)
0,CARLOW,2020,82.361667,10.258333,8.033333
1,CARLOW,2021,78.605000,10.366667,6.916667
2,CARLOW,2022,82.992593,10.633333,7.275000
3,CARLOW,2023,93.320833,11.141667,7.400000
4,CARLOW,2024,75.085106,10.541667,7.216667


In [40]:
# Set workspace
arcpy.env.workspace = gdb_path

In [41]:
# Verify paths
print(f"Shapefile path: {county_shapefile}")
print(f"CSV path: {weather_csv}")
print(f"Shapefile exists: {os.path.exists(county_shapefile)}")
print(f"CSV exists: {os.path.exists(weather_csv)}")


Shapefile path: C:\Users\asus\Desktop\Project\Data\counties\Counties_-_OSi_National_Statutory_Boundaries_-_2019\Counties___OSi_National_Statutory_Boundaries_.shp
CSV path: C:\Users\asus\Desktop\Project\Data\county_weather.csv
Shapefile exists: True
CSV exists: True


In [44]:
# 3. Convert CSVs to geodatabase tables
weather_table = "weather_table"
yearly_weather_table="yearly_weather_table"
arcpy.TableToTable_conversion(weather_csv, gdb_path, weather_table)
arcpy.TableToTable_conversion(yearly_weather_csv, gdb_path, yearly_weather_table)


<Result 'C:\\Users\\asus\\Desktop\\Project\\WebGisProject\\MyProject8\\MyProject8.gdb\\yearly_weather_table'>

In [ ]:
#JOINING WEATHER_TABLE (CONTAINS FIVE-YEAR AVERAGE RAIN, WIND AND TEMPERATURE DATA FROM 2020-2021 ) TO COUNTIES
# Set variables
county_fc = "Counties___OSi_National_Statutory_Boundaries_"  
output_fc = "Counties_with_Weather"  
weather_table = "weather_table"
fields_to_join = ["Year", "Avg_rain_mm_", "Avg_temp__C_", "Avg_wind_knots_"]

# 2. Create a clean copy (preserves original)
arcpy.CopyFeatures_management(county_fc, output_fc)

# Perform join
arcpy.management.JoinField(
    in_data=county_fc,
    in_field="COUNTY",               
    join_table=weather_table,        
    join_field="COUNTY",             
    fields=["Year", "Avg_rain_mm_", "Avg_temp__C_", "Avg_wind_knots_"] 
)

print(f"Join complete. Output: {output_fc}")

In [ ]:
print(f"Absolute path: {os.path.abspath(output_fc)}")

In [86]:
#CREATING YEARLY WEATHER TABLE (CONTAINS YEARLY AVERAGRE(2020,2021,2022,2023,2024) RAIN, WIND AND TEMPERATURE DATA )
# Set variables
county_fc = "Counties___OSi_National_Statutory_Boundaries_" 
output_fc = "Counties_avg_temp"  
weather_table = "yearly_weather_table"  
join_field = "COUNTY"  
fields_to_join = ["Year", "Avg_temp__C_"]  

# 1. Create a clean copy of the original counties
arcpy.CopyFeatures_management(county_fc, output_fc)

# 2. Check if output exists and delete if needed
if arcpy.Exists(output_fc):
    arcpy.Delete_management(output_fc)
arcpy.CopyFeatures_management(county_fc, output_fc)

# 3. Use MakeQueryTable to preserve all matching records
query_table = "in_memory/weather_join"
arcpy.MakeQueryTable_management(
    [output_fc, weather_table],
    query_table,
    "USE_KEY_FIELDS",
    where_clause=f"{output_fc}.{join_field} = {weather_table}.{join_field}"
)

# 4. Export the result to preserve all records
final_output = "Counties_with_all_years"
arcpy.CopyFeatures_management(query_table, final_output)

# 5. Clean up
arcpy.Delete_management("in_memory/weather_join")

<Result 'true'>

In [71]:
#READS AND DISPLAYS Counties_with_all_years average
# Get field names first
fields = [f.name for f in arcpy.ListFields("Counties_with_all_years") if f.type != 'Geometry']
# Read with column names
pd.DataFrame(
    [row for row in arcpy.da.SearchCursor("Counties_with_all_years", fields)],
    columns=fields
).head()

,OBJECTID_1,OBJECTID,CO_ID,ENGLISH,GAEILGE,LOGAINM_ID,GUID,CONTAE,COUNTY,PROVINCE,CENTROID_X,CENTROID_Y,AREA,SHAPE_Leng,OBJECTID_12,COUNTY_1,Year,Avg_rain_mm_,Avg_temp__C_,Avg_wind_knots_,Shape_Length,Shape_Area
0,1,1,50000,DONEGAL,Dún na nGall,100013,2ae19629-1452-13a3-e055-000000000001,Dún na nGall,DONEGAL,Ulster,607296.31,911848.45,4.860754e+09,1.859119e+06,25,DONEGAL,2020,154.058482,10.050000,11.008333,1.859119e+06,4.860754e+09
1,2,1,50000,DONEGAL,Dún na nGall,100013,2ae19629-1452-13a3-e055-000000000001,Dún na nGall,DONEGAL,Ulster,607296.31,911848.45,4.860754e+09,1.859119e+06,26,DONEGAL,2021,117.281991,10.250000,9.383333,1.859119e+06,4.860754e+09
2,3,1,50000,DONEGAL,Dún na nGall,100013,2ae19629-1452-13a3-e055-000000000001,Dún na nGall,DONEGAL,Ulster,607296.31,911848.45,4.860754e+09,1.859119e+06,27,DONEGAL,2022,124.872277,10.500000,10.275000,1.859119e+06,4.860754e+09
3,4,1,50000,DONEGAL,Dún na nGall,100013,2ae19629-1452-13a3-e055-000000000001,Dún na nGall,DONEGAL,Ulster,607296.31,911848.45,4.860754e+09,1.859119e+06,28,DONEGAL,2023,143.542786,10.958333,9.883333,1.859119e+06,4.860754e+09
4,5,1,50000,DONEGAL,Dún na nGall,100013,2ae19629-1452-13a3-e055-000000000001,Dún na nGall,DONEGAL,Ulster,607296.31,911848.45,4.860754e+09,1.859119e+06,29,DONEGAL,2024,116.608057,10.316667,10.208333,1.859119e+06,4.860754e+09


In [87]:
#REMOVES DUPLICATE COLUMNS AND SAVES ONLY SELECTED ONES
# 1. First properly read the feature class into a DataFrame
feature_class = "Counties_with_all_years"

# Get all field names (excluding geometry)
fields = [f.name for f in arcpy.ListFields(feature_class) if f.type != 'Geometry']

# Create DataFrame from SearchCursor
data = []
with arcpy.da.SearchCursor(feature_class, fields) as cursor:
    for row in cursor:
        data.append(row)
        
df = pd.DataFrame(data, columns=fields)

# 2. Now filter the columns
keep_cols = [
    'COUNTY', 'Year', 'Avg_temp__C_', 'Avg_rain_mm_', 'Avg_wind_knots_',
    'CENTROID_X', 'CENTROID_Y', 'Shape_Area'
]

# Only keep columns that actually exist in the DataFrame
available_cols = [col for col in keep_cols if col in df.columns]
clean_df = df[available_cols]


In [88]:
clean_df.head()

,COUNTY,Year,Avg_temp__C_,Avg_rain_mm_,Avg_wind_knots_,CENTROID_X,CENTROID_Y,Shape_Area
0,DONEGAL,2020,10.050000,154.058482,11.008333,607296.31,911848.45,4.860754e+09
1,DONEGAL,2021,10.250000,117.281991,9.383333,607296.31,911848.45,4.860754e+09
2,DONEGAL,2022,10.500000,124.872277,10.275000,607296.31,911848.45,4.860754e+09
3,DONEGAL,2023,10.958333,143.542786,9.883333,607296.31,911848.45,4.860754e+09
4,DONEGAL,2024,10.316667,116.608057,10.208333,607296.31,911848.45,4.860754e+09


In [89]:
#Creating a county shapefile with yearly temperature from year 2020-2024
# 1. creates a proper field mappings object
field_mappings = arcpy.FieldMappings()

# 2. Adds only the fields to keep
fields_to_keep = ["COUNTY", "Year", "CENTROID_X", "CENTROID_Y", "Shape_Area","Avg_temp__C_"]

for field in fields_to_keep:
    fm = arcpy.FieldMap()
    fm.addInputField("Counties_with_all_years", field)
    field_mappings.addFieldMap(fm)

# 3. Creates the temperature shapefile
arcpy.FeatureClassToFeatureClass_conversion(
    "Counties_with_all_years",  
    counties_with_weather,      
    "counties_yearly_temp.shp", 
    where_clause="",           
    field_mapping=field_mappings
)


<Result 'C:\\Users\\asus\\Desktop\\Project\\Data\\counties_with_weather\\counties_yearly_temp.shp'>

In [91]:
#Creating a county shapefile with yearly rain from year 2020-2024
# 1. First creates a proper field mappings object
field_mappings = arcpy.FieldMappings()

# 2. Adds only the fields to keep
fields_to_keep = ["COUNTY", "Year", "CENTROID_X", "CENTROID_Y", "Shape_Area", "Avg_rain_mm_"]

for field in fields_to_keep:
    fm = arcpy.FieldMap()
    fm.addInputField("Counties_with_all_years", field)
    field_mappings.addFieldMap(fm)

# 3. Creates the temperature shapefile
arcpy.FeatureClassToFeatureClass_conversion(
    "Counties_with_all_years",  
    counties_with_weather,      
    "counties_yearly_rain.shp", 
    where_clause="",           
    field_mapping=field_mappings
)


<Result 'C:\\Users\\asus\\Desktop\\Project\\Data\\counties_with_weather\\counties_yearly_rain.shp'>

In [90]:
#Creating a county shapefile with yearly wind from year 2020-2024
# 1. First creates a proper field mappings object
field_mappings = arcpy.FieldMappings()

# 2. Adds only the fields we want to keep
fields_to_keep = ["COUNTY", "Year", "CENTROID_X", "CENTROID_Y", "Shape_Area", "Avg_wind_knots_"]

for field in fields_to_keep:
    fm = arcpy.FieldMap()
    fm.addInputField("Counties_with_all_years", field)
    field_mappings.addFieldMap(fm)

# 3.Creates the temperature shapefile
arcpy.FeatureClassToFeatureClass_conversion(
    "Counties_with_all_years",  
    counties_with_weather,      
    "counties_yearly_wind.shp", 
    where_clause="",           
    field_mapping=field_mappings
)


<Result 'C:\\Users\\asus\\Desktop\\Project\\Data\\counties_with_weather\\counties_yearly_wind.shp'>

In [93]:
#Change coordinate system of all the weather shapefiles
# Set workspace to your counties_with_weather folder
workspace = os.path.join(project_root, "Data", "counties_with_weather")
arcpy.env.workspace = workspace
# Define the output coordinate system (WGS84)
wgs84 = arcpy.SpatialReference(4326)  # WGS84 coordinate system

# List all shapefiles to reproject
shapefiles = [
    "counties_with_weather.shp",
    "counties_yearly_temp.shp",
    "counties_yearly_rain.shp",
    "counties_yearly_wind.shp"
]

# Reproject each shapefile
for shp in shapefiles:
    if arcpy.Exists(shp):
        # Create output name (appends '_WGS84' to original name)
        output_shp = os.path.splitext(shp)[0] + "_WGS84.shp"
        
        # Perform projection
        arcpy.Project_management(
            shp,
            output_shp,
            wgs84
        )
        print(f"Reprojected {shp} to WGS84 as {output_shp}")
    else:
        print(f"Warning: {shp} not found in {workspace}")

Reprojected counties_with_weather.shp to WGS84 as counties_with_weather_WGS84.shp
Reprojected counties_yearly_temp.shp to WGS84 as counties_yearly_temp_WGS84.shp
Reprojected counties_yearly_rain.shp to WGS84 as counties_yearly_rain_WGS84.shp
Reprojected counties_yearly_wind.shp to WGS84 as counties_yearly_wind_WGS84.shp
